In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

sys.path.append('c:\\Projects\\taxi_predicts\\src\\')

In [3]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-03-20 09:00:00')


In [4]:
from inference import load_batch_of_features_from_store

#loading batch of features from store
features = load_batch_of_features_from_store(current_date)

c:\Users\dpetrik\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-02-21 09:00:00 to 2024-03-20 08:00:00


Finished: Reading data from Hopsworks, using Hive (60.41s) 
ts_data=               pickup_hour  rides  pickup_location_id
0      2024-03-02 08:00:00      0                 187
1      2024-02-22 23:00:00      0                 242
2      2024-02-24 17:00:00      0                 195
3      2024-03-01 22:00:00      0                 207
4      2024-02-29 10:00:00      0                 258
...                    ...    ...                 ...
184435 2024-03-20 07:00:00      1                  69
184436 2024-03-20 00:00:00     50                 163
184437 2024-03-19 22:00:00      0                  30
184438 2024-03-19 14:00:00      0                 165
184439 2024-03-19 22:00:00     44                 140

[178080 rows x 3 columns]


In [5]:
from inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.6693931619382161, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6693931619382161
[LightGBM] [Warning] bagging_fraction is set=0.2097048168865949, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2097048168865949


In [11]:
from feature_store_api import get_feature_store

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name='model_predictions_feature_group',
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

NameError: name 'FeatureGroupConfig' is not defined

In [ ]:
#we put the predictions into feature store so they can be later used 
feature_group.insert(predictions, write_options={"wait_for_job": False})

In [ ]:
#we need to run this everytime the feature pipeline completes so we create one more action 